In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install schedule

In [ ]:
!pip install trax

In [ ]:
import json
import random
import numpy as np
from termcolor import colored
import trax   
from trax import layers as tl
from trax.supervised import training
!pip list | grep trax

trax                          1.3.9                


In [ ]:
DATA_FILE = "data.json"
DATA_FILE

'data.json'

In [ ]:
DATA_DIR = "/content/drive/MyDrive/Multilingual_ChatBot/Dataset"


In [ ]:
DIALOGUE_DB = {}


In [ ]:
VOCAB_FILE = "en_32k.subword"

In [ ]:
VOCAB_FILE = "en_32k.subword"

In [ ]:
VOCAB_DIR = "/content/drive/My Drive/Multilingual_ChatBot/coursera-natural-language-processing-specialization/4 - Natural Language Processing with Attention Models/Week 4/data/vocabs"

In [ ]:
def load_json(directory, file):
    with open(f'{directory}/{file}') as file: 
        db = json.load(file)
    return db

DIALOGUE_DB = load_json(DATA_DIR, DATA_FILE)


In [ ]:
print(f'The number of dialogues is: {len(DIALOGUE_DB)}')


The number of dialogues is: 10438


In [ ]:
print(list(DIALOGUE_DB.keys())[0:7])


['SNG01856.json', 'SNG0129.json', 'PMUL1635.json', 'MUL2168.json', 'SNG0073.json', 'SNG01445.json', 'MUL2105.json']


In [ ]:
print(DIALOGUE_DB['SNG0073.json'].keys())

dict_keys(['goal', 'log'])


In [ ]:
DIALOGUE_DB['SNG0073.json']['goal']

{'attraction': {},
 'hospital': {},
 'hotel': {},
 'message': ["You want to book a <span class='emphasis'>taxi</span>. The taxi should go to <span class='emphasis'>pizza hut fen ditton</span> and should depart from <span class='emphasis'>saint john's college</span>",
  "The taxi should <span class='emphasis'>leave after 17:15</span>",
  "Make sure you get <span class='emphasis'>car type</span> and <span class='emphasis'>contact number</span>"],
 'police': {},
 'restaurant': {},
 'taxi': {'fail_info': {},
  'info': {'departure': "saint john's college",
   'destination': 'pizza hut fen ditton',
   'leaveAt': '17:15'},
  'reqt': ['car type', 'phone']},
 'train': {}}

In [ ]:
DIALOGUE_DB['SNG0073.json']['log'][0]

{'metadata': {},
 'text': "I would like a taxi from Saint John's college to Pizza Hut Fen Ditton."}

In [ ]:
print(' Person 1: ', DIALOGUE_DB['SNG0073.json']['log'][0]['text'])
print(' Person 2: ',DIALOGUE_DB['SNG0073.json']['log'][1]['text'])


 Person 1:  I would like a taxi from Saint John's college to Pizza Hut Fen Ditton.
 Person 2:  What time do you want to leave and what time do you want to arrive by?


In [ ]:
def get_conversation(file, data_db):
    result = ''    
    len_msg_log = len(data_db[file]['log'])    
    delimiter_1 = ' Person 1: '
    delimiter_2 = ' Person 2: '
    for i in range(len_msg_log):            
        cur_log = data_db[file]['log'][i]['text']        
        if i % 2 == 0:                   
            result += delimiter_1
        else: 
            result += delimiter_2        
        result += cur_log
    return result


In [ ]:
import numpy as np
import trax
def test_get_conversation(target):

    data = {'file1.json': {'log':[{'text': 'hi'},
                                  {'text': 'hello'},
                                  {'text': 'nice'}]},
            'file2.json':{'log':[{'text': 'a b'}, 
                                 {'text': ''}, 
                                 {'text': 'good '}, 
                                 {'text': 'no?'}]}}    
    res1 = target('file1.json', data)
    res2 = target('file2.json', data)
    
    expected1 = ' Person 1: hi Person 2: hello Person 1: nice'
    expected2 = ' Person 1: a b Person 2:  Person 1: good  Person 2: no?'

    success = 0
    fails = 0    
    try:
        assert res1 == expected1
        success += 1
    except ValueError:
        print('Error in test 1 \nResult  : ', res1, 'x \nExpected: ', expected1)
        fails += 1
    try:
        assert res2 == expected2
        success += 1
    except:
        print('Error in test 2 \nResult  : ', res2, ' \nExpected: ', expected2)
        fails += 1
            
    if fails == 0:
        print("\033[92m All tests passed")
    else:
        print('\033[92m', success," Tests passed")
        print('\033[91m', fails, " Tests failed")
def test_reversible_layer_forward(target):
    f1 = lambda x: x + 2
    g1 = lambda x: x * 3    
    f2 = lambda x: x + 1
    g2 = lambda x: x * 2    
    input_vector1 = np.array([1, 2, 3, 4, 5, 6, 7, 8])
    expected1 = np.array([8, 10, 12, 14, 29, 36, 43, 50])
    
    input_vector2 = np.array([1] * 128)
    expected2 = np.array([3] * 64 + [7] * 64)
    
    success = 0
    fails = 0
    try:
        res = target(input_vector1, f1, g1)
        assert isinstance(res, np.ndarray)
        success += 1
    except:
        print('Wrong type! Output is not of type np.ndarray')
        fails += 1
    try:
        res = target(input_vector1, f1, g1)
        assert np.allclose(res, expected1)
        success += 1
    except ValueError:
        print('Error in test 1 \nResult  : ', res, 'x \nExpected: ', expected1)
        fails += 1
    try:
        res = target(input_vector2, f2, g2)
        assert np.allclose(res, expected2)
        success += 1
    except:
        print('Error in test 2 \nResult  : ', res, ' \nExpected: ', expected2)
        fails += 1            
    if fails == 0:
        print("\033[92m All tests passed")
    else:
        print('\033[92m', success," Tests passed")
        print('\033[91m', fails, " Tests failed")
def test_reversible_layer_reverse(target):    
    f1 = lambda x: x + 2
    g1 = lambda x: x * 3    
    f2 = lambda x: x + 1
    g2 = lambda x: x * 2    
    input_vector1 = np.array([1, 2, 3, 4, 5, 6, 7, 8])
    expected1 = np.array([-3,  0,  3,  6,  2,  0, -2, -4])    
    input_vector2 = np.array([1] * 128)
    expected2 = np.array([1] * 64 + [-1] * 64)    
    success = 0
    fails = 0
    try:
        res = target(input_vector1, f1, g1)
        assert isinstance(res, np.ndarray)
        success += 1
    except:
        print('Wrong type! Output is not of type np.ndarray')
        fails += 1
    try:
        res = target(input_vector1, f1, g1)
        assert np.allclose(res, expected1)
        success += 1
    except ValueError:
        print('Error in test 1 \nResult  : ', res, 'x \nExpected: ', expected1)
        fails += 1
    try:
        res = target(input_vector2, f2, g2)
        assert np.allclose(res, expected2)
        success += 1
    except:
        print('Error in test 2 \nResult  : ', res, ' \nExpected: ', expected2)
        fails += 1
            
    if fails == 0:
        print("\033[92m All tests passed")
    else:
        print('\033[92m', success," Tests passed")
        print('\033[91m', fails, " Tests failed")
        

def test_ReformerLM(target):
    test_cases = [
                {
                    "name":"layer_len_check",
                    "expected":11,
                    "error":"We found {} layers in your model. It should be 11.\nCheck the LSTM stack before the dense layer"
                },
                {
                    "name":"simple_test_check",
      "expected":"Serial[ShiftRight(1)Embedding_train_512DropoutPositionalEncodingDup_out2ReversibleSerial_in2_out2[ReversibleHalfResidualV2_in2_out2[Serial[LayerNorm]SelfAttention]ReversibleSwap_in2_out2ReversibleHalfResidualV2_in2_out2[Serial[LayerNormDense_2048DropoutFastGeluDense_512Dropout]]ReversibleSwap_in2_out2ReversibleHalfResidualV2_in2_out2[Serial[LayerNorm]SelfAttention]ReversibleSwap_in2_out2ReversibleHalfResidualV2_in2_out2[Serial[LayerNormDense_2048DropoutFastGeluDense_512Dropout]]ReversibleSwap_in2_out2]Concatenate_in2LayerNormDropoutDense_trainLogSoftmax]",
                    "error":"The ReformerLM is not defined properly."
                }
            ]
    temp_model = target('train')
    
    success = 0
    fails = 0
    
    for test_case in test_cases:
        try:
            if test_case['name'] == "simple_test_check":
                assert test_case["expected"] == str(temp_model).replace(' ', '').replace('\n','')
                success += 1
            if test_case['name'] == "layer_len_check":
                if test_case["expected"] == len(temp_model.sublayers):
                    success += 1
                else:
                    print(test_case["error"].format(len(temp_model.sublayers))) 
                    fails += 1
        except:
            print(test_case['error'])
            fails += 1
            
    if fails == 0:
        print("\033[92m All tests passed")
    else:
        print('\033[92m', success," Tests passed")
        print('\033[91m', fails, " Tests failed")
def test_tasks(train_task, eval_task):
    target = train_task
    success = 0
    fails = 0     
    try:
        strlabel = str(target._labeled_data)
        assert ("generator" in strlabel) and ("add_loss_weights" in  strlabel)
        success += 1
    except:
        fails += 1
        print("Wrong labeled data parameter in train_task")    
    try:
        strlabel = str(target._loss_layer)
        assert(strlabel == "CrossEntropyLoss_in3")
        success += 1
    except:
        fails += 1
        print("Wrong loss functions. CrossEntropyLoss_in3 was expected")        
    try:
        assert(isinstance(target.optimizer, trax.optimizers.adam.Adam))
        success += 1
    except:
        fails += 1
        print("Wrong optimizer")        
    try:
        assert(isinstance(target._lr_schedule,trax.supervised.lr_schedules._BodyAndTail))
        success += 1
    except:
        fails += 1
        print("Wrong learning rate schedule type")    
    try:
        assert(target._n_steps_per_checkpoint==10)
        success += 1
    except:
        fails += 1
        print("Wrong checkpoint step frequency")        
    target = eval_task
    try:
        strlabel = str(target._labeled_data)
        assert ("generator" in strlabel) and ("add_loss_weights" in  strlabel)
        success += 1
    except:
        fails += 1
        print("Wrong labeled data parameter in eval_task")    
    try:
        strlabel = str(target._metrics).replace(' ', '')
        assert(strlabel == "[CrossEntropyLoss_in3,Accuracy_in3]")
        success += 1
    except:
        fails += 1
        print(f"Wrong metrics. found {strlabel} but expected [CrossEntropyLoss_in3,Accuracy_in3]")                
    if fails == 0:
        print("\033[92m All tests passed")
    else:
        print('\033[92m', success," Tests passed")
        print('\033[91m', fails, " Tests failed")


In [ ]:
import w4_unittest
w4_unittest.test_get_conversation(get_conversation)

 All tests passed


In [ ]:
file = 'SNG01856.json'
conversation = get_conversation(file, DIALOGUE_DB)
print(conversation)

 Person 1: am looking for a place to to stay that has cheap price range it should be in a type of hotel Person 2: Okay, do you have a specific area you want to stay in? Person 1: no, i just need to make sure it's cheap. oh, and i need parking Person 2: I found 1 cheap hotel for you that includes parking. Do you like me to book it? Person 1: Yes, please. 6 people 3 nights starting on tuesday. Person 2: I am sorry but I wasn't able to book that for you for Tuesday. Is there another day you would like to stay or perhaps a shorter stay? Person 1: how about only 2 nights. Person 2: Booking was successful.
Reference number is : 7GAWK763. Anything else I can do for you? Person 1: No, that will be all. Good bye. Person 2: Thank you for using our services.


In [ ]:
def print_conversation(conversation):
    
    delimiter_1 = 'Person 1: '
    delimiter_2 = 'Person 2: '
    
    split_list_d1 = conversation.split(delimiter_1)
    
    for sublist in split_list_d1[1:]:
        split_list_d2 = sublist.split(delimiter_2)
        print(colored(f'Person 1: {split_list_d2[0]}', 'red'))
        
        if len(split_list_d2) > 1:
            print(colored(f'Person 2: {split_list_d2[1]}', 'green'))

            
print_conversation(conversation)


Person 1: am looking for a place to to stay that has cheap price range it should be in a type of hotel 
Person 2: Okay, do you have a specific area you want to stay in? 
Person 1: no, i just need to make sure it's cheap. oh, and i need parking 
Person 2: I found 1 cheap hotel for you that includes parking. Do you like me to book it? 
Person 1: Yes, please. 6 people 3 nights starting on tuesday. 
Person 2: I am sorry but I wasn't able to book that for you for Tuesday. Is there another day you would like to stay or perhaps a shorter stay? 
Person 1: how about only 2 nights. 
Person 2: Booking was successful.
Reference number is : 7GAWK763. Anything else I can do for you? 
Person 1: No, that will be all. Good bye. 
Person 2: Thank you for using our services.


In [ ]:
DIALOGUE_DB['SNG01856.json']['log'][0]


{'metadata': {},
 'text': 'am looking for a place to to stay that has cheap price range it should be in a type of hotel'}

In [ ]:
attraction_file = open('/content/drive/MyDrive/Multilingual_ChatBot/Dataset/attraction_db.json')
attractions = json.load(attraction_file)
print(attractions[0])


{'address': 'pool way, whitehill road, off newmarket road', 'area': 'east', 'entrance fee': '?', 'id': '1', 'location': [52.208789, 0.154883], 'name': 'abbey pool and astroturf pitch', 'openhours': '?', 'phone': '01223902088', 'postcode': 'cb58nt', 'pricerange': '?', 'type': 'swimmingpool'}


In [ ]:
hospital_file = open('/content/drive/MyDrive/Multilingual_ChatBot/Dataset/hospital_db.json')
hospitals = json.load(hospital_file)
print(hospitals[0])


{'department': 'neurosciences critical care unit', 'id': 0, 'phone': '01223216297'}


In [ ]:
hotel_file = open('/content/drive/MyDrive/Multilingual_ChatBot/Dataset/hotel_db.json')
hotels = json.load(hotel_file)
print(hotels[0]) 


{'address': '124 tenison road', 'area': 'east', 'internet': 'yes', 'parking': 'no', 'id': '0', 'location': [52.1963733, 0.1987426], 'name': 'a and b guest house', 'phone': '01223315702', 'postcode': 'cb12dp', 'price': {'double': '70', 'family': '90', 'single': '50'}, 'pricerange': 'moderate', 'stars': '4', 'takesbookings': 'yes', 'type': 'guesthouse'}


In [ ]:
police_file = open('/content/drive/MyDrive/Multilingual_ChatBot/Dataset/police_db.json')
police = json.load(police_file)
print(police[0])



{'name': 'Parkside Police Station', 'address': 'Parkside, Cambridge', 'id': 0, 'phone': '01223358966'}


In [ ]:
restaurant_file = open('/content/drive/MyDrive/Multilingual_ChatBot/Dataset/restaurant_db.json')
restaurants = json.load(restaurant_file)
print(restaurants[0])


{'address': 'Regent Street City Centre', 'area': 'centre', 'food': 'italian', 'id': '19210', 'introduction': 'Pizza hut is a large chain with restaurants nationwide offering convenience pizzas pasta and salads to eat in or take away', 'location': [52.20103, 0.126023], 'name': 'pizza hut city centre', 'phone': '01223323737', 'postcode': 'cb21ab', 'pricerange': 'cheap', 'type': 'restaurant'}


In [ ]:
with open('/content/drive/MyDrive/Multilingual_ChatBot/Dataset/README.json') as file:
    print(file.read())


#####################################################
#####################################################
#  Copyright Cambridge Dialogue Systems Group, 2018 #
#####################################################
#####################################################

Dataset contains the following json files:
1. data.json: the woz dialogue dataset, which contains the conversation  users and wizards, as well as a set of coarse labels for each user turn. Files with multi-domain dialogues have "MUL" in their names. Single domain dialogues have either "SNG" or "WOZ" in their names.
2. restaurant_db.json: the Cambridge restaurant database file, containing restaurants in the Cambridge UK area and a set of attributes.
3. attraction_db.json: the Cambridge attraction database file, contining attractions in the Cambridge UK area and a set of attributes.
4. hotel_db.json: the Cambridge hotel database file, containing hotels in the Cambridge UK area and a set of attributes.
5. train_db.json: th

In [ ]:
all_files = DIALOGUE_DB.keys()
untokenized_data = []
for file in all_files:
    result = get_conversation(file, DIALOGUE_DB)    
    untokenized_data.append(result)
print(untokenized_data[0])


 Person 1: am looking for a place to to stay that has cheap price range it should be in a type of hotel Person 2: Okay, do you have a specific area you want to stay in? Person 1: no, i just need to make sure it's cheap. oh, and i need parking Person 2: I found 1 cheap hotel for you that includes parking. Do you like me to book it? Person 1: Yes, please. 6 people 3 nights starting on tuesday. Person 2: I am sorry but I wasn't able to book that for you for Tuesday. Is there another day you would like to stay or perhaps a shorter stay? Person 1: how about only 2 nights. Person 2: Booking was successful.
Reference number is : 7GAWK763. Anything else I can do for you? Person 1: No, that will be all. Good bye. Person 2: Thank you for using our services.


In [ ]:
random.shuffle(untokenized_data)
cut_off = int(len(untokenized_data) * .05)
train_data, eval_data = untokenized_data[:-cut_off], untokenized_data[-cut_off:]
print(f'number of conversations in the data set: {len(untokenized_data)}')
print(f'number of conversations in train set: {len(train_data)}')
print(f'number of conversations in eval set: {len(eval_data)}')


number of conversations in the data set: 10438
number of conversations in train set: 9917
number of conversations in eval set: 521


In [ ]:
def stream(data):
    while True:
        d = random.choice(data)        
        yield (d, d)

In [ ]:
data_pipeline = trax.data.Serial(
    trax.data.Shuffle(),    
    trax.data.Tokenize(vocab_dir=VOCAB_DIR,
                       vocab_file=VOCAB_FILE),    
    trax.data.FilterByLength(2048),    
    trax.data.BucketByLength(boundaries=[128, 256,  512, 1024],
                             batch_sizes=[16,    8,    4,   2, 1]),
        trax.data.AddLossWeights(id_to_mask=0)
)
train_stream = data_pipeline(stream(train_data))
eval_stream = data_pipeline(stream(eval_data))


In [ ]:
inp, _, _ = next(train_stream)
print("input shape: ", inp.shape)
print(trax.data.detokenize(inp[0], vocab_dir=VOCAB_DIR, vocab_file=VOCAB_FILE))

input shape:  (4, 512)
 Person 1: I am looking for information and could use your help finding a train. Person 2: I'd be happy to help. Which day would you like to travel? Person 1: I need to leave stevenage on saturday. Person 2: I would be happy to help with your request, but first I will need to know what time you'd like to leave/arrive at. Person 1: I don't have a departure time preference but I would like to arrive by 15:45. Person 2: There are five trains to Cambridge on Saturday- the latest arrives at 14:43. Would you like me to book it for you? Person 1: What is the price for that one? Person 2: The cost for the train leaving Stevenage at 13:54, arriving Cambridge at 14:43 is 10.24 pounds, would you like me to book that for you today? Person 1: what's the train ID? Person 2: TR6607. Would you like me to book you on? Person 1: I also need a hotel. I don't care about parking but it does need free wifi.  Person 2: What area would you like to stay in? Person 1: the south . and  mak

In [ ]:
def reversible_layer_forward(x, f, g):
    x1, x2 = np.split(x, 2, axis=-1)         
    y1 = x1 + f(x2)    
    y2 = x2 + g(y1)    
    y = np.concatenate((y1,y2), axis = -1)    
    return y

In [ ]:
from trax import layers as tl
from trax.fastmath import numpy as fastnp
from trax.supervised import training

w4_unittest.test_reversible_layer_forward(reversible_layer_forward)

 All tests passed


In [ ]:
def reversible_layer_reverse(y, f, g):
    y1, y2 = np.split(y, 2, axis=-1)
    x2 = y2 - g(y1)
    x1 = y1 - f(x2)
    x = np.concatenate((x1,x2),axis = -1)
    return x

In [ ]:
w4_unittest.test_reversible_layer_reverse(reversible_layer_reverse)


 All tests passed


In [ ]:
f = lambda x: x + 2
g = lambda x: x * 3
input_vector = np.random.uniform(size=(32,))
output_vector = reversible_layer_forward(input_vector, f, g)
reversed_vector = reversible_layer_reverse(output_vector, f, g)
assert np.allclose(reversed_vector, input_vector)

In [ ]:
f = lambda x: x + np.random.uniform(size=x.shape)
output_vector = reversible_layer_forward(input_vector, f, g)
reversed_vector = reversible_layer_reverse(output_vector, f, g)
assert not np.allclose(reversed_vector, input_vector)  # Fails!!
random_seed = 27686
rng = trax.fastmath.random.get_prng(random_seed)
f = lambda x: x + trax.fastmath.random.uniform(key=rng, shape=x.shape)
output_vector = reversible_layer_forward(input_vector, f, g)
reversed_vector = reversible_layer_reverse(output_vector, f, g)
assert np.allclose(reversed_vector, input_vector,  atol=1e-07)

In [ ]:
def ReformerLM(vocab_size=33000, n_layers=2, mode='train', attention_type=tl.SelfAttention):
    model = trax.models.reformer.ReformerLM( 
        vocab_size=vocab_size,
        n_layers=n_layers,
        mode=mode,
        attention_type=attention_type
    )    
    return model

In [ ]:
temp_model = ReformerLM('train')
print(str(temp_model))
del temp_model

Serial[
  Serial[
    ShiftRight(1)
  ]
  Embedding_train_512
  Dropout
  Serial[
    PositionalEncoding
  ]
  Dup_out2
  ReversibleSerial_in2_out2[
    ReversibleHalfResidualDecoderAttn_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidualDecoderFF_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        Serial[
          FastGelu
        ]
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidualDecoderAttn_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidualDecoderFF_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        Serial[
          FastGelu
        ]
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
  ]
  Concatenate_in2
  LayerNorm
  Dropout
  Serial[
    Dense_train
  ]
]


In [ ]:
w4_unittest.test_ReformerLM(ReformerLM)


We found 10 layers in your model. It should be 11.
Check the LSTM stack before the dense layer
The ReformerLM is not defined properly.
 0  Tests passed
 2  Tests failed


In [ ]:
def training_loop(ReformerLM, train_gen, eval_gen, output_dir = "/content/drive/MyDrive/Multilingual_ChatBot/coursera-natural-language-processing-specialization/4 - Natural Language Processing with Attention Models/Week 4/model"):
    lr_schedule = trax.lr.warmup_and_rsqrt_decay(
        n_warmup_steps=1000, max_value=0.01)
    train_task = training.TrainTask(            
        labeled_data=train_gen,
        loss_layer=tl.CrossEntropyLoss(),
        optimizer=trax.optimizers.Adam(0.01),
        lr_schedule=lr_schedule,
        n_steps_per_checkpoint=10
    )
    eval_task = training.EvalTask(                      
        labeled_data=eval_gen,
        metrics=[tl.CrossEntropyLoss(), tl.Accuracy()]
    )

    loop = training.Loop(ReformerLM(mode='train'),
                         train_task,
                         eval_tasks=[eval_task],
                         output_dir=output_dir)
    return loop


In [ ]:
test_loop = training_loop(ReformerLM, train_stream, eval_stream)
train_task = test_loop.tasks
eval_task = test_loop.eval_tasks

print(train_task)
print(eval_task)


/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:304: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  "jax.host_id has been renamed to jax.process_index. This alias "
/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:317: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  "jax.host_count has been renamed to jax.process_count. This alias "


In [ ]:
w4_unittest.test_tasks(train_task, eval_task)

Wrong labeled data parameter in train_task
Wrong loss functions. CrossEntropyLoss_in3 was expected
Wrong optimizer
Wrong learning rate schedule type
Wrong checkpoint step frequency
Wrong labeled data parameter in eval_task


In [ ]:
!rm -f model/model.pkl.gz
loop = training_loop(ReformerLM, train_stream, eval_stream)
loop.run(5)


/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:304: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  "jax.host_id has been renamed to jax.process_index. This alias "
/usr/local/lib/python3.7/dist-packages/jax/lib/xla_bridge.py:317: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  "jax.host_count has been renamed to jax.process_count. This alias "


In [ ]:
def attention(*args, **kwargs):
    kwargs['predict_mem_len'] = 120
    kwargs['predict_drop_len'] = 120
    return tl.SelfAttention(*args, **kwargs)

model = ReformerLM(
    vocab_size=33000,
    n_layers=7,
    mode='predict',
    attention_type=attention,
)
shape11 = trax.shapes.ShapeDtype((11, 11), dtype=np.int32)

In [ ]:
#model.init_from_file('/content/drive/MyDrive/Multilingual_ChatBot/coursera-natural-language-processing-specialization/4 - Natural Language Processing with Attention Models/Week 4/model/model.pkl.gz',
      #               weights_only=True, input_signature=shape11)

# save the starting state
#STARTING_STATE = model.state

In [ ]:
STARTING_STATE = model.state

In [ ]:
def attention(*args, **kwargs):
    kwargs['predict_mem_len'] = 120
    kwargs['predict_drop_len'] = 120
    return tl.SelfAttention(*args, **kwargs)
model = ReformerLM(
    vocab_size=33000,
    n_layers=6,
    mode='predict',
    attention_type=attention,
)
shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)


In [ ]:
#model.init_from_file('model/model.pkl.gz',
#                     weights_only=True, input_signature=shape11)
#STARTING_STATE = model.state

In [ ]:
def tokenize(sentence, vocab_file, vocab_dir):
    return list(trax.data.tokenize(iter([sentence]), vocab_file=vocab_file, vocab_dir=vocab_dir))[0]

def detokenize(tokens, vocab_file, vocab_dir):
    return trax.data.detokenize(tokens, vocab_file=vocab_file, vocab_dir=vocab_dir)

In [ ]:
def tokenize(sentence, vocab_file, vocab_dir):
    return list(trax.data.tokenize(iter([sentence]), vocab_file=vocab_file, vocab_dir=vocab_dir))[0]

def detokenize(tokens, vocab_file, vocab_dir):
    return trax.data.detokenize(tokens, vocab_file=vocab_file, vocab_dir=vocab_dir)


In [ ]:
def ReformerLM_output_gen(ReformerLM, start_sentence, vocab_file, vocab_dir, temperature):
    input_tokens = tokenize(start_sentence, vocab_file=vocab_file, vocab_dir=vocab_dir)
    input_tokens_with_batch = np.array(input_tokens)[None, :]
    output_gen = trax.supervised.decoding.autoregressive_sample_stream( 
        ReformerLM,
        inputs=input_tokens_with_batch,
        temperature=temperature
    )
    return output_gen

In [ ]:
#import pickle
#WEIGHTS_FROM_FILE = ()
#with open('/content/drive/MyDrive/Multilingual_ChatBot/coursera-natural-language-processing-specialization/4 - Natural Language Processing with Attention Models/Week 4/weights', 'rb') as file:
#    WEIGHTS_FROM_FILE = pickle.load(file)
#shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)
#def attention(*args, **kwargs):
#    kwargs['predict_mem_len'] = 120
#   kwargs['predict_drop_len'] = 120
#    return tl.SelfAttention(*args, **kwargs)
#test_model = ReformerLM(vocab_size=5, n_layers=1, mode='predict', attention_type=attention)
#3test_output_gen = ReformerLM_output_gen(test_model, "test", vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, temperature=0)
#test_model.init_weights_and_state(shape11)
#test_model.weights = WEIGHTS_FROM_FILE
#output = []
#for i in range(6):
#    output.append(next(test_output_gen)[0])
#print(output)
#del test_model 
#del WEIGHTS_FROM_FILE
#del test_output_gen

In [ ]:
shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)
def attention(*args, **kwargs):
    kwargs['predict_mem_len'] = 120
    kwargs['predict_drop_len'] = 120
    return tl.SelfAttention(*args, **kwargs)
model = ReformerLM(
    vocab_size=33000,
    n_layers=6,
    mode='predict',
    attention_type=attention,
)

In [ ]:
#model.init_from_file('model/model.pkl.gz',
#                     weights_only=True, input_signature=shape11)
#STARTING_STATE = model.state

In [ ]:
def generate_dialogue(ReformerLM, model_state, start_sentence, vocab_file, vocab_dir, max_len, temperature):
    delimiter_1 = 'Person 1: ' 
    delimiter_2 = 'Person 2: '
    sentence = ''
    counter = 0
    result = [tokenize(': ', vocab_file=vocab_file, vocab_dir=vocab_dir)]
    ReformerLM.state = model_state
    output = ReformerLM_output_gen(ReformerLM, start_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, temperature=temperature)
    print(start_sentence.split(delimiter_2)[0].strip())
    for o in output:
        result.append(o)
        sentence = detokenize(np.concatenate(result, axis=0), vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)        
        if sentence.endswith(delimiter_1):
            sentence = sentence.split(delimiter_1)[0]
            print(f'{delimiter_2}{sentence}')
            sentence = ''
            result.clear()        
        elif sentence.endswith(delimiter_2):
            sentence = sentence.split(delimiter_2)[0]
            print(f'{delimiter_1}{sentence}')
            sentence = ''
            result.clear()
        counter += 1        
        if counter > max_len:
            break

In [ ]:
#sample_sentence = ' Person 1: Are there theatres in town? Person 2: '
#generate_dialogue(ReformerLM=model, model_state=STARTING_STATE, start_sentence=sample_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, max_len=120, temperature=0.2)

In [ ]:
#sample_sentence = ' Person 1: Is there a hospital nearby? Person 2: '
#generate_dialogue(ReformerLM=model, model_state=STARTING_STATE, start_sentence=sample_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, max_len=120, temperature=0.2)

In [ ]:
#sample_sentence = ' Person 1: Can you book a taxi? Person 2: '
#generate_dialogue(ReformerLM=model, model_state=STARTING_STATE, start_sentence=sample_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, max_len=120, temperature=0.2)

In [ ]:
!git clone https://github.com/parulnith/Building-a-Simple-Chatbot-in-Python-using-NLTK

Cloning into 'Building-a-Simple-Chatbot-in-Python-using-NLTK'...
remote: Enumerating objects: 95, done.
remote: Total 95 (delta 0), reused 0 (delta 0), pack-reused 95
Unpacking objects: 100% (95/95), done.


In [ ]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')


In [ ]:
!pip install nltk


In [ ]:
f=open('/content/Building-a-Simple-Chatbot-in-Python-using-NLTK/chatbot.txt','r',errors = 'ignore')
raw=f.read()
raw = raw.lower()# converts to lowercase


In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:


sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words


In [ ]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


In [ ]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)


In [ ]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response


In [ ]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")


ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
